In [17]:
import librosa
import numpy as np
from dtw import dtw
import os
import sys

# --- FONCTION D'EXTRACTION ---
def extraire_empreinte_complete(chemin_audio):
    """
    Charge un audio et extrait son empreinte vocale (MFCC+Deltas).
    Renvoie un tableau (Temps, Features) -> (N, 39)
    """
    print(f"  -> Traitement de : {chemin_audio}")
    try:
        # Charger le signal (resample par défaut à 22050 Hz)
        signal, sr = librosa.load(chemin_audio, sr=None) # sr=None garde le sample rate original
        
        # Extraire MFCC
        mfccs = librosa.feature.mfcc(y=signal, n_mfcc=13, sr=sr)
        
        # Calculer les Deltas
        delta_mfcc = librosa.feature.delta(mfccs)
        delta2_mfcc = librosa.feature.delta(mfccs, order=2)

        empreinte_complete = np.concatenate((mfccs, delta_mfcc, delta2_mfcc), axis=0)

        return empreinte_complete.T
    
    except Exception as e:
        print(f"ERREUR en traitant {chemin_audio}: {e}")
        return None

# --- FONCTION DE COMPARAISON ---
def comparer_empreintes(emp1, emp2):
    """
    Compare deux empreintes (N, 39) avec DTW et renvoie un score de distance.
    Plus le score est bas, plus c'est similaire.
    """
    if np.std(emp1) > 0:
        emp1_norm = (emp1 - np.mean(emp1)) / np.std(emp1)
    else:
        emp1_norm = emp1
        
    if np.std(emp2) > 0:
        emp2_norm = (emp2 - np.mean(emp2)) / np.std(emp2)
    else:
        emp2_norm = emp2
    
    # Calculer la distance DTW
    alignement = dtw(emp1_norm, emp2_norm, 
                     keep_internals=True, 
                     step_pattern='asymmetric', # Méthode standard
                     open_end=True,            # Permet des longueurs
                     open_begin=True)          # différentes
    
    # Renvoie la distance normalisée
    return alignement.normalizedDistance

def main():

    AUDIO_REFERENCE = "samples/p09/JIM_1.wav"
    AUDIO_TEST      = "samples/p09/JIM_2.wav"

    # Vérifier si les fichiers existent
    if not os.path.exists(AUDIO_REFERENCE) or not os.path.exists(AUDIO_TEST):
        print("\nERREUR : Un des fichiers n'a pas été trouvé.")
        print("Erreur de chemin ?")
        sys.exit(1) 

    # --- Extraire les "tableaux de valeurs" ---
    print("\nExtraction des données")
    empreinte_ref = extraire_empreinte_complete(AUDIO_REFERENCE)
    empreinte_test = extraire_empreinte_complete(AUDIO_TEST)

    if empreinte_ref is None or empreinte_test is None:
        print("erreur d'extraction.")
        sys.exit(1)
        
    print(f"  Forme Réf  : {empreinte_ref.shape}")
    print(f"  Forme Test : {empreinte_test.shape}")

    # --- Comparer ---
    print("\nCalcul de la similarité (DTW)")
    distance = comparer_empreintes(empreinte_ref, empreinte_test)
    
    SEUIL_DE_SIMILARITE = 1.2
    
    if distance < SEUIL_DE_SIMILARITE:
        print("C'est la même personne.")
    else:
        print("Ce sont deux personnes différentes.")
if __name__ == "__main__":
    main()


Extraction des données
  -> Traitement de : samples/p09/JIM_1.wav
  -> Traitement de : samples/p09/JIM_2.wav
  Forme Réf  : (175, 39)
  Forme Test : (176, 39)

Calcul de la similarité (DTW)
C'est la même personne.
